In [59]:
%config InlineBackend.figure_format = 'retina'

In [60]:
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import threading
import warnings
from typing import Tuple
threading.stack_size(67108864)

67108864

# **Solving Systems of Linear Algebraic Equations**

## **0. Functional and helpers**

In [61]:
def get_true_x(n: int) -> np.array:
    assert n > 0, "Parameter n should be positive integer"
    return np.arange(1, n + 1)

In [62]:
def get_matrix_cond(A: np.array) -> float:
    svd = np.linalg.svd(A, compute_uv=False)
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=RuntimeWarning)
        cond = np.fabs(np.max(svd) / np.min(svd))
        return cond if cond != float("inf") else -1

In [63]:
def generate_matrix(n: int, k: float) -> np.array:
    A = np.random.choice([-4., -3., -2., -1., 0.], size=(n, n))
    np.fill_diagonal(A, 0)
    diagonal = np.zeros(n)
    for i in range(n):
        diagonal[i] = np.sum(abs(A[i, :]))
    diagonal[0] += 10 ** -k
    np.fill_diagonal(A, diagonal)
    return np.array(A) if get_matrix_cond(A) != -1 else generate_matrix(n, k)

In [64]:
def get_hilbert_matrix(k: int) -> np.array:
    return np.array([[1 / (i + j + 1) for j in range(k)] for i in range(k)])

In [65]:
def solve_for_matrix_with_method(n: int, k: float, method) -> np.array:
    A = generate_matrix(n, k)
    b = A @ get_true_x(len(A))
    print(f'Solving:\n{A} @ x = {b}\nx = {method(A, b)}')

## **1. Implement Gauss method with leading element selection**

АЛГЕБРАИЧЕСКОЕ ОПИСАНИЕ МЕТОДА ГАУССА (ЧТО ДЕЛАЕМ НА КАЖДОЙ ИТЕРАЦИИ), ПРЯМОГО И ОБРАТНОГО ХОДА. ВЫДВИНУТЬ ТРЕБОВАНИЯ К МАТРИЦЕ, ЧТОБЫ СИСТЕМА БЫЛА РАЗРЕШИМА

https://orion1401.gitbooks.io/numerical_analysys_python/content/metod_gaussa_s_postolbtsovim_viborom_glavnogo_elem.html

In [66]:
def solve_gauss_lead_selection(A: np.array, b: np.array) -> np.array:
    # selecting leading element
    for i in range(n := len(A)):
        pivot_row = i + np.argmax(np.abs(A[i:, i]))
        if i != pivot_row:
            A[[i, pivot_row]], b[[i, pivot_row]] = A[[pivot_row, i]], b[[pivot_row, i]]
        # converting matrix to upper-triangular form
        for iter in range(i + 1, n):
            u = -A[iter][i] / A[i][i]
            A[iter, i:] += u * A[i, i:]
            b[iter] += u * b[i]
    # reverse gauss
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - A[i, i + 1:] @ x[i + 1:]) / A[i, i]
    return x  

In [67]:
solve_for_matrix_with_method(n=8, k=3, method=solve_gauss_lead_selection)

Solving:
[[22.001 -4.    -4.    -2.    -4.    -2.    -2.    -4.   ]
 [-3.    12.     0.    -3.    -1.    -2.    -3.     0.   ]
 [-3.    -3.    18.    -4.    -4.     0.     0.    -4.   ]
 [-1.    -1.     0.    14.    -4.    -3.    -4.    -1.   ]
 [-1.     0.    -2.    -2.    10.    -1.    -1.    -3.   ]
 [ 0.    -3.     0.     0.    -1.     9.    -1.    -4.   ]
 [-2.    -1.     0.    -1.    -2.    -4.    12.    -2.   ]
 [-2.     0.    -4.    -2.    -1.    -2.     0.    11.   ]] @ x = [-83.999 -29.    -23.    -21.     -2.      4.     26.     49.   ]
x = [1. 2. 3. 4. 5. 6. 7. 8.]


## **2. Implement LU-decomposition method**

ОПИСАНИЕ ПРИНЦИПА LU РАЗЛОЖЕНИЯ. АЛГЕБРАИЧЕСКИ ДОКАЗАТЬ ВЕРНОСТЬ

https://orion1401.gitbooks.io/numerical_analysys_python/content/lu_razlozhenie.html

In [68]:
def get_lu_decomposition(A: np.array) -> Tuple[np.array, np.array]:
    n = len(A)
    L, U = np.zeros([n, n]), np.zeros([n, n])
    for i in range(n):
        for k in range(i, n):
            U[i][k] = A[i][k] - sum([L[i][j] * U[j][k] for j in range(i)])
        for k in range(i, n):
            L[k][i] = (A[k][i] - sum([L[k][j] * U[j][i] for j in range(i)])) / U[i][i] if i != k else 1.0
    return L, U

def get_lu_decomposition_vectorized(A: np.array) -> Tuple[np.array, np.array]:
    L = np.eye(n := len(A))
    for i in range(n):
        L[i + 1:, i] = A[i + 1:, i] / A[i, i]
        A[i + 1:, i:] -= L[i + 1:, i, None] * A[i, i:]
    return L, A

In [69]:
test_matrix = np.array([[1., 2., 3.], [4., 5., 6.], [7., 8., 9.]])
test_L, test_U = get_lu_decomposition(test_matrix)
assert np.allclose(test_L @ test_U, test_matrix), "wrong implementation of LU-decomposition"

ОПИСАНИЕ МЕТОДА РЕШЕНИЯ СЛАУ МЕТОДОМ LU РАЗЛОЖЕНИЯ, ОТСЫЛКА К МЕТОДУ ГАУССА

In [70]:
def solve_lu_decomposition(A: np.array, b: np.array) -> np.array:
    L, U = get_lu_decomposition(A)
    y = np.zeros(n := len(A))
    for i in range(n):
        y[i] = b[i] - L[i, :i] @ y[:i]
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - U[i, i + 1:] @ x[i + 1:]) / U[i, i]
    return x

In [71]:
solve_for_matrix_with_method(n=8, k=3, method=solve_lu_decomposition)

Solving:
[[14.001 -1.    -4.     0.    -3.    -1.    -1.    -4.   ]
 [-4.    15.    -2.    -4.    -4.     0.     0.    -1.   ]
 [-1.    -2.    10.    -2.    -1.    -4.     0.     0.   ]
 [-3.     0.    -3.    16.    -3.    -3.    -3.    -1.   ]
 [-3.    -3.    -4.    -4.    24.    -3.    -4.    -3.   ]
 [-1.    -4.    -1.    -3.    -1.    15.    -1.    -4.   ]
 [-4.    -1.    -3.     0.     0.    -4.    15.    -3.   ]
 [-4.    -1.    -4.    -3.    -1.     0.    -4.    17.   ]] @ x = [-59.999 -24.    -12.    -10.     13.     22.     42.     73.   ]
x = [1. 2. 3. 4. 5. 6. 7. 8.]


## **3. Implement iterative Jacobi method**

ОПИСАНИЕ МЕТОДА ЯКОБИ

In [210]:
def solve_jacobi(A: np.array, b: np.array, max_iterations=1e9, eps=1e-8) -> np.array: 
    diag_inv = np.zeros([n := len(A), n])
    np.fill_diagonal(diag_inv, [val ** -1 if val != 0 else eps for val in np.diag(A)])
    np.fill_diagonal(A, 0)
    A = diag_inv @ A
    b = diag_inv @ b
    x = np.zeros(n)
    for iter in range(int(max_iterations)):
        x_curr = b - A @ x
        if np.linalg.norm(x_curr - x) < eps:
            break
        x = x_curr
    print(f"\nJacobi computing finished in: {iter} iterations\n")
    return x

In [211]:
solve_for_matrix_with_method(n=4, k=4, method=solve_jacobi)


Jacobi computing finished in: 2314927 iterations

Solving:
[[ 9.0001 -4.     -2.     -3.    ]
 [ 0.      2.     -2.      0.    ]
 [-3.     -1.      5.     -1.    ]
 [-4.     -1.     -3.      8.    ]] @ x = [-16.9999  -2.       6.      17.    ]
x = [0.99841785 1.99841784 2.99841784 3.99841784]
